In [31]:
import keras
keras.__version__
import pandas as pd
import copy
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
# %matplotlib inline

from keras.models import Sequential 
from keras.layers import Dense

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_score
import tensorflow as tf


In [32]:
# !pip install -q xlrd

In [33]:
# from google.colab import drive
# drive.mount('/content/drive')

In [34]:

# dfferdig = pd.read_excel("drive/My Drive/DAT300/fullgraminor2018.xlsx", sheet_name=0)

# dfferdig.set_index('26-06-18Unnamed: 0', inplace=True)

In [35]:
# Define the directory with data files
path = r"C:\Users\fahad\Documents\Master Thesis\Phenotyping\Data_from_2017_and_2018"
path = path.replace("\\", "/") + '/'

In [36]:
dfferdig = pd.read_csv(path + 'Graminor_2018_complete.csv', delimiter=';', encoding="ISO-8859-1")

dfferdig.set_index('26-06-18Unnamed: 0', inplace=True)

In [37]:

features = ['26-06-18_bluemedian', '26-06-18_redmedian', '26-06-18_mtci', 'MAT',	
       '02-07-18_mtci', '02-07-18_evi', '19-07-18_bluemedian',	
       '19-07-18_greenmedian', '19-07-18_redmedian', '19-07-18_nirmedian',	
       '19-07-18_mtci']


df_x = dfferdig[features]
df_y = dfferdig['26-06-18GrainYield']

In [38]:
df_x.shape

(599, 11)

In [39]:
scaler= StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(
        df_x, df_y, test_size=0.3, random_state=1)
X_train.iloc[:, :] = scaler.fit_transform(X_train)
X_test.iloc[:, :] = scaler.transform(X_test)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [40]:
def build_model(act, lay, nur):
    
    # Input: 
    #     act: which activations function, denoted by 'name_of_activation' i.e. 'relu'
    #     lay: integer, number of layers 
    #     nur: number of neurons within each layer. possibly we would want a different number of neurons per layer within a model. 
    #          but for now we keep the same number of neurons for all layers
    
    model = Sequential()
    model.add(Dense(nur, activation=act, input_shape=(X_train.shape[1],)))
    n=1
    while n <= lay:
      model.add(Dense(nur, activation=act))
      n += 1
    
    model.add(Dense(1, activation=act))
    model.compile(optimizer='rmsprop', loss=[tf.keras.metrics.MeanSquaredError()], metrics=[tf.keras.metrics.MeanAbsoluteError()])
    
    return model

In [41]:
def K_fold(X_train, y_train, model_activation, model_layers, model_neurons, k, num_epochs, history=False):
  """Custom function for K-fold Cross validation for both keras 
  Inspired by the Colab notebook 'DAT300 - 05 - regression example.ipynb'"""
  
  num_val_samples = X_train.shape[0] // k
  all_scores = np.zeros(k)
  all_mae_histories = []

  for i in range(k):
      print('processing fold #', i+1)

      # Prepare the validation data: data from partition # k
      val_data = X_train[i * num_val_samples: (i + 1) * num_val_samples]
      val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]

      # Prepare the training data: data from all other partitions
      partial_train_data = np.concatenate(
          [X_train[:i * num_val_samples],
           X_train[(i + 1) * num_val_samples:]],
          axis=0)

      partial_train_targets = np.concatenate(
          [y_train[:i * num_val_samples],
           y_train[(i + 1) * num_val_samples:]],
          axis=0)

      # Build the Keras model (already compiled)
      model = build_model(model_activation, model_layers, model_neurons)

      # Train the model (in silent mode, verbose=0)
      history = model.fit(partial_train_data, partial_train_targets,
                          validation_data=(val_data, val_targets),
                          epochs=num_epochs, batch_size=1, verbose=0)

      # Evaluate the model on the validation data
      val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
      all_scores[i] = val_mae
      
      mae_history = history.history['val_mean_absolute_error']
      all_mae_histories.append(mae_history)
  
  average_mae_history = [
      np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]
  print(min(average_mae_history))
  if history:
    return average_mae_history
  else:
    return all_scores

In [42]:
def plot_epochs(mae_history, title):
  plt.plot(range(1, len(mae_history) + 1), mae_history)
  plt.title(title)
  plt.xlabel('Epochs')
  plt.ylabel('Validation MAE')
  plt.show()
  

## result table


In [43]:
K_fold_results = pd.DataFrame({'Model': ['ReLU','Deep ReLU','selu', 'Deep selu', 'LinReg Quadratic', 'LinReg Cubic'],
                      'Validation MAE': np.nan,
                      'Hyperparameters': np.nan})
K_fold_results.set_index('Model', inplace=True)
K_fold_results = K_fold_results[['Validation MAE', 'Hyperparameters']]

K_fold_results

,Validation MAE,Hyperparameters
Model,,
ReLU,NaN,NaN
Deep ReLU,NaN,NaN
selu,NaN,NaN
Deep selu,NaN,NaN
LinReg Quadratic,NaN,NaN
LinReg Cubic,NaN,NaN


## ReLU

In [44]:
relu_layers = 2
relu_neurons = 8
relu_history = K_fold(X_train=X_train,
                      y_train=y_train,
                      model_activation='relu',
                      model_layers=relu_layers,
                      model_neurons=relu_neurons,
                      k=10,
                      num_epochs=100,
                      history=True)
print(min(relu_history))
plot_epochs(relu_history, 'ReLU')

processing fold # 1


AttributeError: in user code:

    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\compile_utils.py:187 __cal

## Deep ReLU

In [ ]:
deep_relu_layers = 5
deep_relu_neurons = 32
deep_relu_history = K_fold(X_train=X_train, y_train=y_train, model_activation='relu', model_layers=deep_relu_layers, model_neurons=deep_relu_neurons, k=10, num_epochs=100, history=True)

plot_epochs(deep_relu_history, 'Deep ReLU')

## Selu

In [ ]:
selu_layers = 2
selu_neurons = 8
selu_history = K_fold(X_train=X_train, y_train=y_train, model_activation='selu', model_layers=selu_layers, model_neurons=selu_neurons, k=10, num_epochs=100, history=True)

plot_epochs(selu_history, 'Selu')

## deep selu


In [ ]:
deep_selu_layers = 5
deep_selu_neurons = 32
deep_selu_history = K_fold(X_train=X_train, y_train=y_train, model_activation='selu', model_layers=deep_selu_layers, model_neurons=deep_selu_neurons, k=10, num_epochs=100, history=True)

plot_epochs(deep_selu_history, 'Deep Selu')

## Linear regression

In [ ]:
# Linear
LinReg_lin_score = cross_val_score(LinearRegression(), X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
K_fold_results.loc['LinReg', :] = [abs(np.mean(LinReg_lin_score)), ' ']

# Quadratic
X_quad = PolynomialFeatures(degree=2).fit_transform(X_train)
LinReg_quad_score = cross_val_score(LinearRegression(), X_quad, y_train, cv=5, scoring='neg_mean_absolute_error')
K_fold_results.loc['LinReg Quadratic', :] = [abs(np.mean(LinReg_quad_score)), ' ']

# Cubic
X_cubic = PolynomialFeatures(degree=3).fit_transform(X_train)
LinReg_cub_score = cross_val_score(LinearRegression(), X_cubic, y_train, cv=5, scoring='neg_mean_absolute_error')
K_fold_results.loc['LinReg Cubic', :] = [abs(np.mean(LinReg_cub_score)), ' ']

In [ ]:
K_fold_results

## Setting best models

In [ ]:
bestdeep = build_model(act='selu', lay=5, nur=32)#set the parameters for the best model
bestdeep.fit(X_train, y_train, epochs=49, batch_size=1, verbose=0)

#bestdeep.predict(X_test)
bestdeep.evaluate(X_test, y_test, batch_size=1)

In [ ]:
bestlin = PolynomialFeatures(degree=3).fit_transform(X_test)
lin_score = cross_val_score(LinearRegression(), bestlin, y_test, cv=5, scoring='neg_mean_absolute_error')
K_fold_results.loc['LinReg Cubic', :] = [abs(np.mean(lin_score)), ' ']

In [0]:
K_fold_results